In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math as math
import matplotlib as mpl
import matplotlib.pyplot as plt

## Distancias

In [3]:
def Dist_Euclidea(x_i, x_r):

        Dist_Euclidea = ( ( x_i - x_r )**2 ).sum()

        Dist_Euclidea = np.sqrt(Dist_Euclidea)

        return Dist_Euclidea

In [4]:
def Matrix_Dist_Euclidea(Data):

    # Paso previo necesario si Data es pd.DataFrame  -->  Data = Data.to_numpy()

    n = len(Data)

    M =  np.empty((n , n))

    
    for i in range(0, n):

         for r in range(0, n):

             if i >= r :
               
                 M[i,r] = 0

             else :

                 M[i,r] = Dist_Euclidea(Data[i,:] , Data[r,:])   

                      
    return M 

In [5]:
def Dist_Minkowski(x_i, x_r, q):

    Dist_Minkowski = ( ( ( abs( x_i - x_r) )**q ).sum() )**(1/q)

    return Dist_Minkowski

In [6]:
def Matrix_Dist_Minkowski(Data, q):

    # Paso previo necesario si Data es pd.DataFrame  -->  Data = Data.to_numpy()

    n = len(Data)

    M =  np.empty((n , n))

    
    for i in range(0, n):

         for r in range(0, n):

             if i >= r :
               
                 M[i,r] = 0

             else :

                 M[i,r] = Dist_Minkowski(Data[i,:] , Data[r,:], q)   

                      
    return M 

In [7]:
def Dist_Canberra(x_i, x_r):

    numerator =  abs( x_i - x_r )

    denominator =  ( abs(x_i) + abs(x_r) )
       
    numerator=np.array([numerator], dtype=float)

    denominator=np.array([denominator], dtype=float)

    # The following code is to eliminate zero division problems

    Dist_Canberra = ( np.divide( numerator , denominator , out=np.zeros_like(numerator), where=denominator!=0) ).sum() 

    return Dist_Canberra

In [8]:
def Matrix_Dist_Canberra(Data):

    # Paso previo necesario si Data es pd.DataFrame  -->  Data = Data.to_numpy()

    n = len(Data)

    M =  np.empty((n , n))

    
    for i in range(0, n):

         for r in range(0, n):

             if i >= r :
               
                 M[i,r] = 0

             else :

                 M[i,r] = Dist_Canberra(Data[i,:] , Data[r,:])   

                      
    return M 

In [9]:
def Dist_Mahalanobis_2(x_i, x_r, S_inv):  # Más eficiente que la anterior

    # All the columns of Data must be type = 'float' or 'int' (specially not 'object'), in other case we will find 
    # dimensional problems when Python compute   x @ S_inv @ x.T

    x = (x_i - x_r)
    
    # x = np.array([x]) 
    
    # En este caso este paso no será necesario por el tipo de estructura Python (una lista) que va a ser un medoid en el
    # algoritmo que a continuación definiremos. De hecho si este paso no se suprimer salta un error al usar la distancia
    # de Mahalanobis, puesto que la operacion x @ S_inv @ x.T no está bien definida.

    Dist_Maha = np.sqrt( x @ S_inv @ x.T )  # x @ S_inv @ x.T = np.matmul( np.matmul(x , S_inv) , x.T )

    Dist_Maha = float(Dist_Maha)

    return Dist_Maha

In [10]:
def Dist_Mahalanobis_3(x, S_inv):  # Más eficiente que la anterior

    # All the columns of Data must be type = 'float' or 'int' (specially not 'object'), in other case we will find 
    # dimensional problems when Python compute   x @ S_inv @ x.T

    # x = (x_i - x_r)

    # x = np.array([x]) # necessary step to transpose a 1D array

    Dist_Maha = np.sqrt( x @ S_inv @ x.T )  # x @ S_inv @ x.T = np.matmul( np.matmul(x , S_inv) , x.T )

    Dist_Maha = float(Dist_Maha)

    return Dist_Maha

In [11]:
def Matrix_Dist_Mahalanobis_3(Data):

    # Paso previo necesario si Data es pd.DataFrame  -->  Data = Data.to_numpy()

    n = len(Data)

    M =  np.empty((n , n))

    S_inv=np.linalg.inv( np.cov(Data , rowvar=False) )

   
    for i in range(0, n):

         for r in range(0, n):

             if i >= r :
               
                 M[i,r] = 0

             else :

                 M[i,r] = Dist_Mahalanobis_3(x = np.array([Data[i,:] - Data[r,:]]) , S_inv=S_inv ) 

                      
    return M 

In [12]:
def a_b_c_d_Matrix(Data):

    X = Data

    a = X @ X.T

    n = X.shape[0]

    p = X.shape[1]

    ones_matrix = np.ones((n, p)) 

    b = (ones_matrix - X) @ X.T

    c = b.T

    d = (ones_matrix - X) @ (ones_matrix - X).T

    return a , b , c , d , p

In [13]:
def alpha(x_i, x_r):

    alpha = sum(x_i == x_r)

    return(alpha)

In [14]:
def Matrix_Gower_Similarity(Data, p1, p2, p3 ):

    # Data tiene que ser un numpy array tal que:
    # las primeras p1 tiene que ser variables cuantitativas, 
    # las siguientes p2 binarias y las restantes p3 multicales

    # Si Data no contiene variables cuantitativas --> p1=0
    # Si Data no contiene variables binarias --> p2=0
    # Si Data no contiene variables multiclase --> p3=0

    n = len(Data)

    M =  np.empty((n , n))

############################################################
    
    G_vector = np.repeat(0.5, p1)

    for k in range(0, p1):

        G_vector[k] = Data[:,k].max() - Data[:,k].min()

############################################################
   
    ones = np.repeat(1, p1)

    Quant_Data = Data[: , 0:p1]

    Binary_Data = Data[: , (p1):(p1+p2)]

    Multiple_Data = Data[: , (p1+p2):(p1+p2+p3) ]

############################################################

    a, b, c, d, p = a_b_c_d_Matrix(Binary_Data)

############################################################

    for i in range(0, n):

         for r in range(0, n):

            if i > r :
               
                 M[i,r] = 0
            
            elif i == r :
               
                 M[i,r] = 1
            
            else :

                # Calculo de la distancia de Gower entre las observaciones x_i y x_r

                numerator_part_1 = ( ones - ( abs(Quant_Data[i,:] - Quant_Data[r,:]) / G_vector ) ).sum() 

                numerator_part_2 = a[i,r] + alpha(Multiple_Data[i,:], Multiple_Data[r,:])

                numerator = numerator_part_1 + numerator_part_2
                

                if p1 + (p2 - d[i,r]) + p3 == 0:

                    M[i,r] = 0

                else:
                    
                    denominator = p1 + (p2 - d[i,r]) + p3

                    M[i,r] = numerator / denominator
                      
   
    return M  

In [15]:
def Matrix_Gower_Distance(Data, p1, p2, p3 ):

    M = Matrix_Gower_Similarity(Data, p1, p2, p3)

    M = M + M.T - np.diag(np.repeat(1 , len(M)), k=0)

    M = np.sqrt( 1 - M )

    return M

## KNN programado de cero

In [131]:
Data = pd.read_csv('House_Price_Regression.csv')

Data_Mixed = Data.loc[:, ['latitude', 'longitude', 'price', 'size_in_m_2', 'balcony_recode', 'private_garden_recode', 'quality_recode']]

Data_Mixed_train = Data_Mixed.sample(frac=0.8, replace=False, weights=None, random_state=222, axis=None, ignore_index=False)

Data_Mixed_new =  Data_Mixed.drop( Data_Mixed_train.index , )

In [132]:
## TRAIN DATA (Datos dispobles --> se usan para entrenar el modelo)

X_train = Data_Mixed_train.loc[: , Data_Mixed_train.columns != 'quality_recode']
Y_train = Data_Mixed_train.loc[: , 'quality_recode']

## NEW DATA (Nuevos datos de los predictores)

X_new = Data_Mixed_new.loc[: , Data_Mixed_test.columns != 'quality_recode']

# En la práctica real no se tienen datos sobre la respuesta asociado a las nuevas obsrvaciones de los predictores
# Eso es justo lo que se quiere predecir.
# Pero en este ejemplo al usar como "nuevos" datos una parte del data set original, si que tenemos esa información.

Y_new = Data_Mixed_new.loc[: , 'quality_recode'] 


In [133]:
X_train

,latitude,longitude,price,size_in_m_2,balcony_recode,private_garden_recode
925,25.083483,55.148688,3195000,233.558142,1.0,0.0
1502,25.065872,55.232983,430000,43.478604,1.0,0.0
867,25.191285,55.275202,3499900,221.016237,1.0,0.0
746,25.083858,55.138714,8000000,240.340061,1.0,0.0
729,25.046296,55.200783,635356,58.435987,1.0,0.0
...,...,...,...,...,...,...
131,25.073593,55.137516,2175000,128.670655,0.0,0.0
1324,25.072573,55.131009,2300000,127.277110,1.0,0.0
1391,25.079895,55.134126,1396000,78.874647,1.0,0.0
1191,25.191719,55.270677,3100000,185.341485,1.0,0.0


In [153]:
len(X_train)

1524

In [134]:
X_new

,latitude,longitude,price,size_in_m_2,balcony_recode,private_garden_recode
6,25.114275,55.139764,8503600,191.565986,0.0,0.0
7,25.114275,55.139764,3119900,94.296545,0.0,0.0
13,25.132519,55.151574,1849000,99.499113,1.0,0.0
15,25.132021,55.151405,2349990,103.029427,1.0,0.0
36,25.073590,55.126352,3599999,182.647298,1.0,0.0
...,...,...,...,...,...,...
1890,25.072954,55.128089,3800000,167.875721,0.0,0.0
1894,25.191107,55.269910,980888,101.078464,1.0,0.0
1900,25.176892,55.310712,1500000,100.985561,1.0,0.0
1901,25.166145,55.276684,1230000,70.606280,1.0,0.0


In [22]:
X = pd.concat([X_train , X_new])
X

,latitude,longitude,price,size_in_m_2,balcony_recode,private_garden_recode
925,25.083483,55.148688,3195000,233.558142,1.0,0.0
1502,25.065872,55.232983,430000,43.478604,1.0,0.0
867,25.191285,55.275202,3499900,221.016237,1.0,0.0
746,25.083858,55.138714,8000000,240.340061,1.0,0.0
729,25.046296,55.200783,635356,58.435987,1.0,0.0
...,...,...,...,...,...,...
1890,25.072954,55.128089,3800000,167.875721,0.0,0.0
1894,25.191107,55.269910,980888,101.078464,1.0,0.0
1900,25.176892,55.310712,1500000,100.985561,1.0,0.0
1901,25.166145,55.276684,1230000,70.606280,1.0,0.0


In [25]:
M_Gower = Matrix_Gower_Distance(Data=X.to_numpy(), p1=4, p2=2, p3=1)
M_Gower

array([[0.        , 0.51145857, 0.52082898, ..., 0.55779441, 0.54651001,
        0.59823175],
       [0.51145857, 0.        , 0.53466633, ..., 0.51292966, 0.48611169,
        0.54305625],
       [0.52082898, 0.53466633, 0.        , ..., 0.47035119, 0.46643008,
        0.61733298],
       ...,
       [0.55779441, 0.51292966, 0.47035119, ..., 0.        , 0.43983966,
        0.60131672],
       [0.54651001, 0.48611169, 0.46643008, ..., 0.43983966, 0.        ,
        0.58190807],
       [0.59823175, 0.54305625, 0.61733298, ..., 0.60131672, 0.58190807,
        0.        ]])

In [154]:
M_Gower_new_data = M_Gower[ len(X_train):len(X) , 0:len(X_train) ]  

# Nos quedamos con las filas de la 1524 a la 1904 , que son las correspondientes a las nuevas observaciones de los predictores.
# Nos queamos con las columnas de la 0 a la 1524, que son las correspondientes a las observaciones de train de los predictores.

# 'M_new_data' contiene las distancias (de Gower) entre las nuevas observaciones de los predictores y las de entrenamiento.

# Su fila i-esima contiene las distancias entre la i-esima nueva observacion de los predictores y las observaciones de entrenamiento.

In [155]:
M_Gower_new_data.shape

(381, 1524)

In [157]:
def KNN(Distance_Matrix_New_Data , k ) :

    # k es el hiper-parametro del algoritmo KNN.

    # 'Distance_Matrix_New_Data' debe contenere las distancias entre las nuevas observaciones de los predictores y las de train.

    # Su fila i-esima debe contener las distancias entre la i-esima nueva observacion de los predictores y las observaciones de train de los predictores.


    Y_predict_x_new_i_LIST = []

    for i in range(0, len(Distance_Matrix_New_Data)):

        distancias_x_new_i = pd.DataFrame({'id_x_train': X_train.index  , 'distancias': Distance_Matrix_New_Data[i,:]})

        distancias_x_new_i_sort = distancias_x_new_i.sort_values(by=["distancias"]).reset_index(drop=True)

        knn_x_new_i = distancias_x_new_i_sort.iloc[0:k , :]

        categorias_knn_x_new_i = []

        
        for j in knn_x_new_i.iloc[:,0]:

            categorias_knn_x_new_i.append(Y_train[j])

    
        unique, counts = np.unique(categorias_knn_x_new_i , return_counts=True)

        unique_Y , counts_Y = np.unique(Y_train , return_counts=True)

    
        if len(unique) == len(unique_Y) :

            proporciones_categorias_knn_x_new_i = pd.DataFrame({'prop_categoria': counts/k, 'categoria': unique_Y })
        
        elif len(unique) < len(unique_Y) :

            proporciones_categorias_knn_x_new_i = pd.DataFrame({'prop_categoria': counts/k, 'categoria': unique })


        Y_predict_x_new_i = proporciones_categorias_knn_x_new_i.sort_values(by=["prop_categoria"], ascending=False).iloc[0,:]['categoria']


        Y_predict_x_new_i_LIST.append(Y_predict_x_new_i)

    
    df_predictions = pd.DataFrame({'id_x_new':range(0, len(M_new_data)) , 'Y_predict': Y_predict_x_new_i_LIST})

    return df_predictions

In [158]:
df_predictions = KNN(Distance_Matrix_New_Data = M_Gower_new_data , k=10)

In [159]:
df_predictions

,id_x_new,Y_predict
0,0,1.0
1,1,1.0
2,2,2.0
3,3,2.0
4,4,2.0
...,...,...
376,376,1.0
377,377,2.0
378,378,2.0
379,379,2.0


In [160]:
Y_new.reset_index().quality_recode

0      0.0
1      2.0
2      2.0
3      2.0
4      1.0
      ... 
376    2.0
377    2.0
378    3.0
379    2.0
380    2.0
Name: quality_recode, Length: 381, dtype: float64

In [165]:
TA = sum( df_predictions.Y_predict == Y_new.reset_index().quality_recode ) / len(Y_new)

TA

0.7795275590551181

In [166]:
TE = 1 - TA
TE

0.22047244094488194

In [167]:
M_Euclidea = Matrix_Dist_Euclidea(Data=X.to_numpy())

In [169]:
M_Euclidea  = M_Euclidea + M_Euclidea.T

In [170]:
M_Euclidea_new_data = M_Euclidea[ len(X_train):len(X) , 0:len(X_train) ]  


In [172]:
df_predictions = KNN(Distance_Matrix_New_Data = M_Euclidea_new_data , k=10)

In [173]:
df_predictions

,id_x_new,Y_predict
0,0,2.0
1,1,2.0
2,2,2.0
3,3,1.0
4,4,2.0
...,...,...
376,376,2.0
377,377,2.0
378,378,2.0
379,379,2.0


In [174]:
TA = sum( df_predictions.Y_predict == Y_new.reset_index().quality_recode ) / len(Y_new)
TA

0.5249343832020997

In [175]:
TE = 1 - TA
TE

0.47506561679790027